In [ ]:
import pandas as pd
from pymongo import MongoClient
import random

from mongo_aggregation_verbs import *

mongo_client = MongoClient('18.236.138.158', 27016)
database_reference = mongo_client.twitter

In [ ]:
database_reference.collection_names()

collection_reference = database_reference.instructor_test_group

test_group = database_reference.instructor_test_group

In [ ]:
match_empty_url_arrays = { MATCH : { "entities.urls" : [] } }

list(test_group.aggregate(
    [
        match_empty_url_arrays,
        { COUNT : "text" }
    ]
))

In [ ]:
job_hashtags = ['job', 'jobs', 'hiring', 'careerarc']
location_hashtags = ['california', 'losangeles', 'la', 'santamonica', 'glendale', 'paloalto']
match_not_in_bad = { MATCH : { "text" : { "$in" : job_hashtags + location_hashtags } } }
project_to_text_keep_id = { PROJECT : { "text" : "$entities.hashtags.text" } }
project_to_id = { PROJECT : { "_id" : 1 } }

bad_ids = list(test_group.aggregate(
    [
        match_non_empty_hashtag_arrays,
        project_to_text_keep_id,
        unwind_text,
        project_to_lower,
        match_not_in_bad,
        project_to_id
    ]
))
bad_ids[:10], len(bad_ids)

In [ ]:
bad_ids = [bad_id['_id'] for bad_id in bad_ids]
bad_ids[:10]

In [ ]:
not_in_bad_ids = { "$nin" : bad_ids }

In [ ]:
not_in_bad_ids_and_no_url = { 
    "_id"           : not_in_bad_ids, 
    "entities.urls" : []
}

just_the_text = {
    "text" : 1,
    "_id"  : 0
}

In [ ]:
test_group.find_one(
    not_in_bad_ids_and_no_url,
    just_the_text
)

In [ ]:
cur  = test_group.find(
    not_in_bad_ids_and_no_url,
    just_the_text
)

tweets = list(cur)
tweet_text = pd.DataFrame(tweets)

In [ ]:
len(tweet_text)

In [ ]:
tweet_text.head()

In [ ]:
tweet_text.text = tweet_text.text.str.replace('http\S+|www.\S+', '', case=False)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(tweet_text.text)
word_occurence = tfidf.transform(tweet_text.text).todense()

In [ ]:
words = tfidf.get_feature_names()
word_sample = random.sample(words, 20)
word_occurence_m = pd.DataFrame(word_occurence, columns=words)
word_occurence_m[word_sample].head()

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
lda = LatentDirichletAllocation(n_topics=10, learning_method='batch')
lda.fit(word_occurence)

In [ ]:
lda_df = pd.DataFrame(lda.components_, columns=words).T

In [ ]:
def filter_topic(lda_df, index, threshold):
    return (lda_df[lda_df[index] > threshold][index]
            .sort_values(ascending=False))

In [ ]:
filter_topic(lda_df, 0, 10)

In [ ]:
filter_topic(lda_df, 1, 10)

In [ ]:
filter_topic(lda_df, 2, 10)

In [ ]:
filter_topic(lda_df, 3, 10)

In [ ]:
filter_topic(lda_df, 4, 10)

In [ ]:
filter_topic(lda_df, 5, 10)

In [ ]:
filter_topic(lda_df, 6, 6)

In [ ]:
filter_topic(lda_df, 7, 7)

In [ ]:
filter_topic(lda_df, 8, 7)

In [ ]:
filter_topic(lda_df, 9, 7)